In [1]:
import pandas as pd
import csv
import pandas_profiling
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
def remove_umlaut(s):
    u = 'ü'.encode()
    U = 'Ü'.encode()
    a = 'ä'.encode()
    A = 'Ä'.encode()
    o = 'ö'.encode()
    O = 'Ö'.encode()
    ss = 'ß'.encode()
    
    s = s.encode()
    s = s.replace(u,b'ue')
    s = s.replace(U, b'Ue')
    s = s.replace(a, b'ae')
    s = s.replace(A, b'Ae')
    s = s.replace(o, b'oe')
    s = s.replace(O, b'Oe')
    s = s.replace(ss, b'ss')

    s = s.decode('utf-8')
    return s  

In [2]:
import json
import ijson
company_authorities = []
file_entityfacts = "../../DataSets/authorities-koerperschaft_lds_20210213.jsonld"
authority_entities = ijson.items(open(file_entityfacts, 'r',encoding='utf-8'),'item')
for ind, authority_entity in enumerate(authority_entities):
    row = []
    ##########name
    value_name = ""
    if authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#preferredNameForTheCorporateBody") is not None:
        value_name = authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#preferredNameForTheCorporateBody")[0].get("@value")
    else:
        value_name = "empty_name_authorities_"+str(ind)
    row.append(value_name)
    ##########address
    value_address = "empty_address_authorities_"+str(ind)
    row.append(value_address)
    #########companyNumber
    value_companyNumber = "empty_companyNumber_authorities"+str(ind)
    row.append(value_companyNumber)
    ############CompanyYear
    value_companyYear = []
    if authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#dateOfEstablishment") is not None:
        value_companyYear = authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#dateOfEstablishment")[0].get("@value")
        if len(value_companyYear) == 4:
            value_companyYear = value_companyYear
        elif len(value_companyYear) == 10:
            value_companyYear = value_companyYear[-4:]
        elif len(value_companyYear) == 8:
            value_companyYear = value_companyYear[0:4]
           # print(value_companyYear)
    else:
        value_companyYear = "0000"
    row.append(value_companyYear)
    ############state
    value_state = "empty_state_authorities_"+str(ind)
    row.append(value_state)
    ###########status 
    value_status = "empty_status_authorities"+str(ind)
    row.append(value_status)
    ############telefon
    telefon = "0000"
    row.append(telefon)
    ############officer
    officer_name = "empty_vname empty_nachname_authotities_"+str(ind)
    position = "empty_position_authorities_"+str(ind)
    row.append(officer_name)
    row.append(position)
    ############oldAuthorities
    value_oldAuthorities = ""
    if authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#oldAuthorityNumber") is not None:
        value_oldAuthorities = authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#oldAuthorityNumber")[0].get('@value')
    else:
        value_oldAuthorities = "empty_value_oldAuthorities_"+str(ind)
    row.append(value_oldAuthorities)
    #####################################
    row.append("company_authorities_id_"+str(ind))
    company_authorities.append(row)

In [3]:
company_authorities

[['International Acetylene Association',
  'empty_address_authorities_0',
  'empty_companyNumber_authorities0',
  '0000',
  'empty_state_authorities_0',
  'empty_status_authorities0',
  '0000',
  'empty_vname empty_nachname_authotities_0',
  'empty_position_authorities_0',
  '(DE-588b)1009-1',
  'company_authorities_id_0'],
 ['Verband Schweizerischer Elektrizitätswerke',
  'empty_address_authorities_1',
  'empty_companyNumber_authorities1',
  '1895',
  'empty_state_authorities_1',
  'empty_status_authorities1',
  '0000',
  'empty_vname empty_nachname_authotities_1',
  'empty_position_authorities_1',
  '(DE-588)1088845797',
  'company_authorities_id_1'],
 ['Kanada. Statistics Canada. Manufacturing and Primary Industries Division',
  'empty_address_authorities_2',
  'empty_companyNumber_authorities2',
  '0000',
  'empty_state_authorities_2',
  'empty_status_authorities2',
  '0000',
  'empty_vname empty_nachname_authotities_2',
  'empty_position_authorities_2',
  '(DE-588b)11296-3',
  'c

In [4]:
company_ocdata = []
generate_id = "company_ocdata_id_"
with open("../../DataSets/de_companies_ocdata.jsonl", encoding='utf-8') as f:
    count = 0
    for line in f:
        row = []
        d = json.loads(line)
        ###################
        value_name = "" 
        value_address = ""
        value_companyNumber = "" 
        value_companyYear = ""
        value_officer = ""
        value_state = ""
        value_status = "" 
        value_officer_name = ""
        value_position = ""
        ################### iterate ele in line
        for ele,value in d.items():
            
            if ele == "all_attributes":
                #print(type(d["all_attributes"]))
                for q,v1 in d["all_attributes"].items():
                    if q == "native_company_number":
                        if v1 is not None:
                            value_companyNumber = "Amtsgericht "+v1
                            #value_companyNumber = remove_umlaut(value_companyNumber)
                        else:
                            value_companyNumber = "empty_companyNumber_ocdata"
                    elif q == "federal_state":
                        if v1 is not None:
                            value_state = v1
                            value_state = remove_umlaut(value_state)
                        else:
                            value_state = "empty_state_ocdata"
                            #print(value_state)
            elif ele == "current_status":
                if value is not None:
                    value_status = value
                    if value_status == "removed":
                        value_status = "INACTIVE"
                    else:
                        value_status = "ACTIVE"
                    #value_status = remove_umlaut(value_status)
                else:
                    value_status = "empty_status_ocdata"
            elif ele == "name":
                if value is not None and value != "" and value.isspace() != True:
                    value_name = value
                    value_name = remove_umlaut(value_name)
                else:
                    value_name = "empty_name_ocdata"
            elif ele == "registered_address":
                if value is not None:
                    value_address = value
                    value_address = remove_umlaut(value_address)
                else:
                    value_address = "empty_address_ocdata"
            elif ele == "officers":
                for key,v2 in d["officers"][0].items():
                    if key == "name":
                        if v2 is not None and v2 != "" and v2.isspace() != True:
                            value_officer_name = v2
                            value_officer_name = remove_umlaut(value_officer_name)
                        else:
                            value_officer_name = "empty_officerName_ocdata_"+str(count)
                    elif key == "position":
                        if v2 is not None:
                            value_position = v2
                            value_position = remove_umlaut(value_position)
                        else:
                            value_position = "empty_position_ocdata_"+str(count)
        row.append(value_name)
        row.append(value_address)
        row.append(value_companyNumber)
        row.append(value_companyYear)
        row.append(value_state)
        row.append(value_status)
        row.append("1000")
        row.append(value_officer_name)
        row.append("empty_position_ocdata_"+str(count))
        row.append("empty_value_ocdata_"+str(count))
        row.append(generate_id+str(count))
        company_ocdata.append(row)
        count+=1

In [5]:
matching_pairs = []
for line_authorities in company_authorities:
    for line_ocdata in company_ocdata:
        if line_authorities[0][0:10] == line_ocdata[0][0:10]:
            matching_pairs.append([line_authorities,line_ocdata])
            break
matching_pairs[0]

[['International Acetylene Association',
  'empty_address_authorities_0',
  'empty_companyNumber_authorities0',
  '0000',
  'empty_state_authorities_0',
  'empty_status_authorities0',
  '0000',
  'empty_vname empty_nachname_authotities_0',
  'empty_position_authorities_0',
  '(DE-588b)1009-1',
  'company_authorities_id_0'],
 ['International Business Communication GmbH',
  'Mergenthaler Allee 77, 65760 Eschborn.',
  'Amtsgericht Frankfurt am Main HRB 52499',
  '',
  'Hesse',
  'INACTIVE',
  '1000',
  'Wolfgang Rautenberg',
  'empty_position_ocdata_1532',
  'empty_value_ocdata_1532',
  'company_ocdata_id_1532']]

In [6]:
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import SubElement
from xml.etree.ElementTree import ElementTree
from xml.dom import minidom

In [7]:
root = Element('companies')
for line in matching_pairs:
    #print(line[8])
    head = SubElement(root,'company',{'id':line[0][10]})
    print(line[0][10])
    #name
    name = SubElement(head,'name')
    name.text = line[0][0]
    #address 
    address = SubElement(head,'address')
    address.text = line[0][1]
    #companyNumber
    companyNumber = SubElement(head,'companyNumber')
    companyNumber.text = line[0][2]
    #Year
    companyYear = SubElement(head,"companyYear")
    companyYear.text = line[0][3]
    #state 
    state = SubElement(head,'state')
    state.text = line[0][4]
    #status
    status = SubElement(head,'status')
    status.text = line[0][5]
    #telefon
    telefon = SubElement(head,'telefon')
    telefon.text = line[0][6]
    #officer
    officers = SubElement(head,'officers')
    officer = SubElement(officers, 'officer')
    officer_name = SubElement(officer, 'officerName')
    officer_name.text = line[0][7]
    position = SubElement(officer, 'position')
    position.text = line[0][8]
    #oldAiuthorities
    oldAuthorities = SubElement(head,'oldAuthorities')
    oldAuthorities.text = line[0][9]
#write out xml data
tree = ElementTree(root)
tree.write('Dataset/Blocking_authorities_ocdata.xml', encoding = 'utf-8')

company_authorities_id_0
company_authorities_id_1
company_authorities_id_7
company_authorities_id_9
company_authorities_id_11
company_authorities_id_12
company_authorities_id_13
company_authorities_id_14
company_authorities_id_15
company_authorities_id_16
company_authorities_id_18
company_authorities_id_20
company_authorities_id_24
company_authorities_id_26
company_authorities_id_30
company_authorities_id_36
company_authorities_id_37
company_authorities_id_39
company_authorities_id_41
company_authorities_id_44
company_authorities_id_46
company_authorities_id_47
company_authorities_id_48
company_authorities_id_49
company_authorities_id_50
company_authorities_id_52
company_authorities_id_53
company_authorities_id_54
company_authorities_id_56
company_authorities_id_57
company_authorities_id_58
company_authorities_id_59
company_authorities_id_61
company_authorities_id_62
company_authorities_id_65
company_authorities_id_66
company_authorities_id_67
company_authorities_id_70
company_authorit

In [8]:
root = Element('companies')
for line in matching_pairs:
    #print(line[8])
    head = SubElement(root,'company',{'id':line[1][10]})
    print(line[1][10])
    #name
    name = SubElement(head,'name')
    name.text = line[1][0]
    #address 
    address = SubElement(head,'address')
    address.text = line[1][1]
    #companyNumber
    companyNumber = SubElement(head,'companyNumber')
    companyNumber.text = line[1][2]
    #Year
    companyYear = SubElement(head,"companyYear")
    companyYear.text = line[1][3]
    #state 
    state = SubElement(head,'state')
    state.text = line[1][4]
    #status
    status = SubElement(head,'status')
    status.text = line[1][5]
    #telefon
    telefon = SubElement(head,'telefon')
    telefon.text = line[1][6]
    #officer
    officers = SubElement(head,'officers')
    officer = SubElement(officers, 'officer')
    officer_name = SubElement(officer, 'officerName')
    officer_name.text = line[1][7]
    position = SubElement(officer, 'position')
    position.text = line[1][8]
    #oldAiuthorities
    oldAuthorities = SubElement(head,'oldAuthorities')
    oldAuthorities.text = line[1][9]
#write out xml data
tree = ElementTree(root)
tree.write('Dataset/Blocking_ocdata_authorities.xml', encoding = 'utf-8')

company_ocdata_id_1532
company_ocdata_id_661015
company_ocdata_id_151507
company_ocdata_id_22174
company_ocdata_id_1532
company_ocdata_id_540437
company_ocdata_id_1532
company_ocdata_id_709
company_ocdata_id_574855
company_ocdata_id_310104
company_ocdata_id_7024
company_ocdata_id_308318
company_ocdata_id_40123
company_ocdata_id_281990
company_ocdata_id_4259547
company_ocdata_id_234935
company_ocdata_id_129
company_ocdata_id_2245
company_ocdata_id_154815
company_ocdata_id_2487707
company_ocdata_id_2187210
company_ocdata_id_838686
company_ocdata_id_334975
company_ocdata_id_305466
company_ocdata_id_573887
company_ocdata_id_896306
company_ocdata_id_1884788
company_ocdata_id_311962
company_ocdata_id_151125
company_ocdata_id_785617
company_ocdata_id_647895
company_ocdata_id_1511765
company_ocdata_id_317678
company_ocdata_id_377470
company_ocdata_id_370451
company_ocdata_id_16916
company_ocdata_id_305850
company_ocdata_id_315061
company_ocdata_id_1546
company_ocdata_id_1016817
company_ocdata_